In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.2.4
2.5


In [3]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [4]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [6]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [7]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'), function=<function sum_over at 0x000001AEB0A9DCA0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'), function=<function sum_over at 0x000001AEB0A9DCA0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'), function=<function sum_over at 0x000001AEB0A9DCA0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'), function=<function sum_over at 0

In [8]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{' 0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.3_DikeIncrease 0':0}),
           Policy('policy 2', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':0,
                                'A.3_DikeIncrease 0':5}),
           Policy('policy 3', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':0,
                                'A.3_DikeIncrease 0':10}),
            Policy('policy 4', **{'0_RfR 0':1,
                                 '0_RfR 1': 0,
                                '0_RfR 2': 0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 5', **{'0_RfR 0':0,
                                 '0_RfR 0':1,
                                '0_RfR 0':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 6', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':1,
                                'A.3_DikeIncrease 0':0}),
        Policy('policy 7', **{' 1_RfR 0':1,
                                  '1_RfR 1':0,
                                  '1_RfR 2':0,
                                  'A.3_DikeIncrease 0':0}),
           Policy('policy 8', **{'1_RfR 0':0,
                                 '1_RfR 1':1,
                                '1_RfR 2':0,
                                'A.5_DikeIncrease 0':0}),
           Policy('policy 9', **{'1_RfR 0':0,
                                 '1_RfR 1':0,
                                '1_RfR 2':1,
                                'A.3_DikeIncrease 0':0}),
            Policy('policy 10', **{'2_RfR 0':1,
                                 '2_RfR 1':0,
                                '2_RfR 2':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 11', **{'2_RfR 0':0,
                                 '2_RfR 1':1,
                                '2_RfR 2':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 12', **{'2_RfR 0':0,
                                 '2_RfR 1':0,
                                '2_RfR 2':1,
                                'A.3_DikeIncrease 0':0})
            ]

In [9]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [10]:
use_pickle1 = True
if use_pickle1:
    with open('data/formulation_results.pickle', 'rb') as filehandler:
        results = pickle.load(filehandler)
else:
    #pass the policies list to EMA workbench experiment runs
    n_scenarios = 100
    with MultiprocessingEvaluator(dike_model) as evaluator:
        results = evaluator.perform_experiments(n_scenarios, policies)
    # Save results in Pickle file
    with open('data/formulation_results.pickle', 'wb') as filehandler:
        pickle.dump(results, filehandler)


In [11]:
import pandas as pd
a,b = results
df = pd.DataFrame(b)
df["policy"] = pd.DataFrame(a)["policy"]
df

#a = np.min(df['A.3 Total Costs'])#



,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,policy
0,9.506030e+07,0.000000,1.118850e+08,0.002483,1.389003e+08,0.231006,2.485405e+07,0.004771,2.983051e+08,0.220732,0.0,0.0,policy 1
1,9.506030e+07,0.000000,7.323512e+08,0.478150,1.959433e+07,0.000000,1.686899e+07,0.000000,7.344192e+07,0.001258,0.0,0.0,policy 1
2,2.512252e+09,1.652505,1.093152e+08,0.000000,9.549722e+07,0.131235,1.686899e+07,0.000000,7.197799e+07,0.000000,0.0,0.0,policy 1
3,9.506030e+07,0.000000,1.260903e+08,0.017792,6.827920e+07,0.097941,4.256294e+07,0.013238,1.166882e+08,0.046773,0.0,0.0,policy 1
4,2.508217e+09,1.649032,1.093152e+08,0.000000,2.421301e+08,0.400379,4.272025e+07,0.011854,7.197799e+07,0.000000,0.0,0.0,policy 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,2.679112e+09,1.903257,1.022563e+08,0.000000,5.437894e+07,0.065691,1.623000e+07,0.000000,4.542000e+07,0.000000,30700000.0,0.0,policy 12
1196,2.508319e+09,1.885683,1.022563e+08,0.000000,1.858000e+07,0.000000,7.595766e+07,0.024761,4.542000e+07,0.000000,30700000.0,0.0,policy 12
1197,8.964065e+07,0.002453,1.070046e+08,0.004018,3.478393e+07,0.025415,4.141514e+07,0.011307,4.542000e+07,0.000000,30700000.0,0.0,policy 12
1198,3.159608e+08,0.177236,2.741476e+08,0.169799,1.858000e+07,0.000000,8.926443e+07,0.036158,4.542000e+07,0.000000,30700000.0,0.0,policy 12


In [12]:
#dit moet dus gebeuren per 100 scenarios
print(np.min(df['A.3 Total Costs']))
print(np.average(df['A.3 Total Costs']))
print(np.max(df['A.3 Total Costs']))
print(np.percentile(df['A.3 Total Costs'], 95))
print(np.percentile(df['A.3 Total Costs'], 5))

18580000.0
338919753.2389691
2785616795.6057982
1954454010.5283349
18580000.0


In [13]:
TCA_3 = df['A.3 Total Costs']
ED_3 = df['A.3_Expected Number of Deaths']
RfR3 = df['RfR Total Costs']
EEC = df['Expected Evacuation Costs']

TCA_3min = np.zeros(12)
TCA_3avg = np.zeros(12)
TCA_3max = np.zeros(12)
TCA_3P5 = np.zeros(12)
TCA_3P95 = np.zeros(12)


for i in range(len(policies)):
    TCA_3min[i] += np.min([TCA_3[i], TCA_3[i+100]])
    TCA_3avg[i] += np.average([TCA_3[i], TCA_3[i+100]])
    TCA_3max[i] += np.max([TCA_3[i], TCA_3[i+100]])
    TCA_3P5[i] += np.percentile([TCA_3[i], TCA_3[i+100]], 5)
    TCA_3P95[i] += np.percentile([TCA_3[i], TCA_3[i+100]], 95)






In [14]:
print(np.min(df['A.3_Expected Number of Deaths']))
print(np.average(df['A.3_Expected Number of Deaths']))
print(np.max(df['A.3_Expected Number of Deaths']))

0.0
0.5433395895860357
3.5261066879565766


In [15]:
print(np.min(df['RfR Total Costs']))
print(np.average(df['RfR Total Costs']))
print(np.max(df['RfR Total Costs']))

0.0
58075000.0
217800000.0


In [16]:
print(np.min(df['Expected Evacuation Costs']))
print(np.average(df['Expected Evacuation Costs']))
print(np.max(df['Expected Evacuation Costs']))

0.0
0.0
0.0
